In [1]:
import sys
sys.path.append("..")

In [2]:
from constants import *
from supporting_functions import *
from des_functions import *
from cryptanalysis import *
from collections import Counter
import numpy as np

In [3]:
key61 = \
'101101' + \
'111011' + \
'xxxxxx' + \
'000111' + \
'001100' + \
'011111' + \
'010001' + \
'110000'

In [4]:
key62 = \
'101101' + \
'111011' + \
'100101' + \
'000111' + \
'001100' + \
'011111' + \
'010001' + \
'xxxxxx'

In [5]:
key6 = ""
for i in range(8):
    if key61[6*i:6*(i+1)]!="xxxxxx":
        key6 += key61[6*i:6*(i+1)]
    else:
        key6 += key62[6*i:6*(i+1)]
key6

'101101111011100101000111001100011111010001110000'

In [6]:
def find_indices_keys(key):
    keys = []
    y = np.array(key)
    c = y[PERMUTED_CHOICE1C]
    d = y[PERMUTED_CHOICE1D]
    for i in range(16):
        c = left_shift_find(c, KEY_SHIFT_SCHEDULE[i])
        d = left_shift_find(d, KEY_SHIFT_SCHEDULE[i])
        bit_string = np.concatenate([c,d])
        y = np.array(list(bit_string))
        round_key = y[PERMUTED_CHOICE2]
        keys.append(round_key)
        
    return keys

def left_shift_find(bit_string, count):
    bit_string = np.concatenate([bit_string[count:], bit_string[:count]])
    return bit_string

In [7]:
key_indices = find_indices_keys(np.arange(0,64))

In [8]:
found_index = []
for a in key_indices[5]:
    found_index.append(int(a))

In [9]:
pending_index = []
for i in range(64):
    if i not in found_index:
        pending_index.append(i)

In [10]:
key = list("x" * 64)
for i in range(48):
    ix = key_indices[5][i]
    key[ix] = key6[i]

In [11]:
found_key = ""
for i in range(64):
    if i%8!=7:
        found_key+=key[i]
found_key

'001110xx1110111100100111x011x11xx0010x010111100010001x00'

In [20]:
print(key)

['0', '0', '1', '1', '1', '0', 'x', 'x', 'x', '1', '1', '1', '0', '1', '1', 'x', '1', '1', '0', '0', '1', '0', '0', 'x', '1', '1', '1', 'x', '0', '1', '1', 'x', 'x', '1', '1', 'x', 'x', '0', '0', 'x', '1', '0', 'x', '0', '1', '0', '1', 'x', '1', '1', '1', '0', '0', '0', '1', 'x', '0', '0', '0', '1', 'x', '0', '0', 'x']


In [29]:
# PLAIN TEXT
plain_texts = []
cipher_texts = []
for i in range(10):
    p = bits_to_str(format(random.randint(0, 2**64-1), '064b'))
    c = des_encode_single(p)
    plain_texts.append(str_to_bits(p))
    cipher_texts.append(str_to_bits(c))

In [39]:
correct_key = None
for i in range(2**8):
    bit_string = format(i, '08b')
    ixp = 0
    ixf = 0
    candidate_key = ""
    for j in range(64):
        if j%8==7:
            continue
        if j in found_index:
            candidate_key+=found_key[ixf]
            ixf+=1
        else:
            candidate_key+=bit_string[ixp]
            ixf+=1
            ixp+=1
    
    # Testing for the key
    correct_key = True
    candidate_key = add_parity_key(candidate_key)
    keys = compute_keys(candidate_key)
    for k in range(len(plain_texts)):
        p = plain_texts[k]
        c = cipher_texts[k]
        produced_c = encode_des(p, keys, 6)
        # print(candidate_key, produced_c)
        if produced_c!=c:
            correct_key = False
            break
        
    if correct_key==True:
        correct_key = candidate_key
        print(candidate_key)

0011101_1111011_1100100_1110011_0110100_1010101_1110001_0001000_
